# INTERACTIVE AFFINITY TOOL  



Running the notebook:  

1. Select "Run all" from the the Runtime drop down menu located in the upper left of notebook  

2. Scroll down the last cell  

3. Use the sliders to select the values for R0max, R1max, and R2max you wish to use  

4. Select the region of interest from the region drop down widget found just below the R3max slider  ]

5. Click on "Run Interact" located below the region drop down menu  

6. Wait for the plot to be generated  

7. Repeat steps 3-6 as desired  


 


## Jefferson Lab Angular Momentum (JAM) Collaboration

In [1]:
!git clone https://github.com/Juniper82/SIDIS-Affinity

import numpy as np
import pandas as pd
from pandas import read_excel 
from copy import deepcopy
from ipywidgets import *
import logging, os 
logging.disable(logging.WARNING) 
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf
print("tf.__version__", tf.__version__)

%cd '/content/SIDIS-Affinity'
from plottools import plotEIC1

%matplotlib inline

Cloning into 'SIDIS-Affinity'...
remote: Enumerating objects: 1331, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 1331 (delta 23), reused 38 (delta 16), pack-reused 1277
Receiving objects: 100% (1331/1331), 697.82 MiB | 30.43 MiB/s, done.
Resolving deltas: 100% (649/649), done.
Checking out files: 100% (981/981), done.
tf.__version__ 2.7.0
/content/SIDIS-Affinity


In [2]:
collinear_region_name = 'collinear'
current_region_name = 'current'
target_region_name = 'target'
TMD_region_name = 'TMD'
soft_region_name = 'soft'
collinear_lable_name = 'collinearaff'
target_lable_name = 'targetaff'
current_lable_name = 'currentaff'
TMD_lable_name = 'tmdaff'
soft_lable_name = 'softaff'

fname = './expdata/eic.xlsx'
data=pd.read_excel(fname,index_col=0)

data.pT.max()
data.keys()

test_features = deepcopy(data)
test_features = test_features.drop(columns=['W2', 'hadron', 'target'])

In [3]:
tmd_model_name = './models/final_%s' % TMD_region_name
tmd_model = tf.keras.models.load_model(tmd_model_name)
target_model_name = './models/final_%s' % target_region_name
target_model = tf.keras.models.load_model(target_model_name)
collinear_model_name = './models/final_%s' % collinear_region_name
collinear_model = tf.keras.models.load_model(collinear_model_name)
current_model_name = './models/final_%s' % current_region_name
current_model = tf.keras.models.load_model(current_model_name)
soft_model_name = './models/final_%s' % soft_region_name
soft_model = tf.keras.models.load_model(soft_model_name)

In [4]:
# Create a slider widget for interactive Rmax selection
R0maxSlider=widgets.FloatSlider(min=.1,max=1.,step=.01,continuous_update=False)
R1maxSlider=widgets.FloatSlider(min=.1,max=1.,step=.01,continuous_update=False)
R2maxSlider=widgets.FloatSlider(min=.1,max=1.,step=.01,continuous_update=False)
regionDrop=widgets.Dropdown(
    options=[('tmd', TMD_lable_name), ('target', target_lable_name), 
             ('collinear', collinear_lable_name), ('current', current_lable_name),
             ('soft', soft_lable_name)],
    value=TMD_lable_name,
    description='Region:')

# Define a function to predict tmdaff that corresponds the widgets Rmax values and return plot
def plot(R0max=R0maxSlider,R1max=R1maxSlider,R2max=R2maxSlider,region=regionDrop):
       
    test_features['R0max'] = np.ones(len(test_features))*R0maxSlider.value
    test_features['R1max'] = np.ones(len(test_features))*R1maxSlider.value
    test_features['R2max'] = np.ones(len(test_features))*R2maxSlider.value

    if region == TMD_lable_name:
        predictions = tmd_model.predict(test_features).flatten()

    elif region == target_lable_name:
        predictions = target_model.predict(test_features).flatten()

    elif region == collinear_lable_name:
        predictions = collinear_model.predict(test_features).flatten()

    elif region == soft_lable_name:
        predictions = soft_model.predict(test_features).flatten()

    else:
        predictions = current_model.predict(test_features).flatten()

    return plotEIC1(data, predictions, hadron = 'pi+', affinity = region, plotx = 'pT', ploty = 'z', cmap_name = 'Spectral', yscale = 'linear')

In [5]:
#Initiate user interaction 
execute = interactive(plot, {'manual': True},R0=R0maxSlider,R1max=R1maxSlider,R2max=R2maxSlider,region=regionDrop)
execute

interactive(children=(FloatSlider(value=0.1, continuous_update=False, description='R0max', max=1.0, min=0.1, s…